<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250531%5D%20FedAvg%20MLP-LSTM_logged_extended%20on%20WSN-BFSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***20250531 FedAvg MLP-LSTM_logged_extended on WSN-BFSF***

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, InputLayer
from keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

In [2]:
!pip install imbalanced-learn

In [3]:
# ----------------------------
# Preprocessing and Partitioning
# ----------------------------
df = pd.read_csv("dataset.csv").dropna()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = LabelEncoder().fit_transform(df[col])

X, y = df.drop('Class', axis=1), df['Class']
X = StandardScaler().fit_transform(X)
X, y = SMOTE(random_state=42).fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [4]:
# ----------------------------
# Federated Setup
# ----------------------------
num_clients = 5
rounds = 30
client_X = np.array_split(X_train, num_clients)
client_y = np.array_split(y_train, num_clients)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_test_cat = to_categorical(y_test)
num_classes = y_test_cat.shape[1]
input_shape = (X_test.shape[1], 1)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [5]:
# ----------------------------
# Model Definition
# ----------------------------
import tensorflow as tf # Import tensorflow
def build_model(input_shape, num_classes):
    model = Sequential([
        InputLayer(input_shape=input_shape),
        LSTM(64, activation='tanh'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# ----------------------------
# Profiling Resources
# ----------------------------
global_model = build_model(input_shape, num_classes)
total_params = global_model.count_params()
model_size_MB = round((total_params * 4) / (1024 ** 2), 2)  # 4 bytes/param
peak_memory_MB = model_size_MB

@tf.function
def model_call(x): return global_model(x)
dummy_input = tf.random.normal([1] + list(input_shape))
try:
    from tensorflow.python.profiler.model_analyzer import profile
    from tensorflow.python.profiler.option_builder import ProfileOptionBuilder
    flops = profile(model_call.get_concrete_function(dummy_input).graph,
                    options=ProfileOptionBuilder.float_operation()).total_float_ops
except:
    flops = -1

print("\n--- Model Resource Profile ---")
print(f"Total Parameters: {total_params}")
print(f"Model Size (MB): {model_size_MB}")
print(f"FLOPs per Inference: {flops}")
print(f"Peak Memory (MB): {peak_memory_MB}")

# ----------------------------
# Training Loop + Comm Cost
# ----------------------------
global_weights = global_model.get_weights()
log = []
comm_cost_per_round_MB = model_size_MB * num_clients  # each client sends full weights

for r in range(rounds):
    round_start = time.time()
    client_weights = []

    for i in range(num_clients):
        local_model = build_model(input_shape, num_classes)
        local_model.set_weights(global_weights)
        X_c = client_X[i].reshape(client_X[i].shape[0], client_X[i].shape[1], 1)
        y_c = to_categorical(client_y[i], num_classes)
        local_model.fit(X_c, y_c, epochs=1, batch_size=32, verbose=0)
        client_weights.append(local_model.get_weights())

    # FedAvg Aggregation
    global_weights = [np.mean([w[i] for w in client_weights], axis=0) for i in range(len(global_weights))]
    global_model.set_weights(global_weights)

    # Evaluation
    y_pred = global_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    acc = accuracy_score(np.argmax(y_test_cat, axis=1), y_pred_classes)
    prec = precision_score(np.argmax(y_test_cat, axis=1), y_pred_classes, average='weighted')
    rec = recall_score(np.argmax(y_test_cat, axis=1), y_pred_classes, average='weighted')
    f1 = f1_score(np.argmax(y_test_cat, axis=1), y_pred_classes, average='weighted')
    auc = roc_auc_score(y_test_cat, y_pred, multi_class='ovr')
    round_time = time.time() - round_start

    log.append({
        'Round': r+1,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'AUC': auc,
        'RoundTime(s)': round_time,
        'CommCost_MB': comm_cost_per_round_MB
    })

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.



--- Model Resource Profile ---
Total Parameters: 33732
Model Size (MB): 0.13
FLOPs per Inference: 67544
Peak Memory (MB): 0.13
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6572/6572 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

6572/6572 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6572/6572 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6572/6572 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step


In [6]:
# ----------------------------
# Save Log
# ----------------------------
df_log = pd.DataFrame(log)
# Correct the filename to match the download call
log_filename = "fedavg_training_and_resource_log_WSN-BFSF.csv"
df_log.to_csv(log_filename, index=False)
print(f"\nLog saved to {log_filename}")


# ----------------------------
# Download Log
# ----------------------------
from google.colab import files
# Use the corrected filename for download
files.download(log_filename)


Log saved to fedavg_training_and_resource_log_WSN-BFSF.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>